<a href="https://colab.research.google.com/github/thmnguyen/team3-project-c/blob/main/llm_QA_Langchain_ollama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This code to generate answer column for Hallubench dataset using Ollama-Mistral model local through Langchain. (to evaluate the answers, use the other notebook: Ragas_eva_Ollama_langchain.ipynb)

In [ ]:
# Install ollama: The Ollama API is now available at 127.0.0.1:11434
! curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
import subprocess
process = subprocess.Popen("ollama serve", shell=True)
# credit: https://stackoverflow.com/questions/78437376/run-ollama-run-llama3-in-colab-raise-err-error-could-not-connect-to-ollama

# download the Mistral model
!ollama pull mistral

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest 
pulling ff82381e2bea...   0% ▕▏    0 B/4.1 GB                  pulling manifest 
pulling ff82381e2bea...   0% ▕▏    0 B/4.1 GB                  pulling manifest 
pulling ff82381e2bea...   0% ▕▏    0 B/4.1 GB                  pulling manifest 
pulling ff82381e2bea...   0% ▕▏    0 B/4.1 GB                  pulling manifest 
pulling ff82381e2bea...   0% ▕▏    0 B/4.1 GB                  pulling manifest 
pulling ff82381e2bea...   0% ▕▏    0 B/4.1 GB                  pulling manifest 
pulling ff82381e2bea.

In [ ]:
# Install necessary packages
!pip install datasets langchain-core langchain-community ragas
!pip install sentence-transformers
!pip install pandas
!pip install langchain
!pip install huggingface_hub
!pip install tiktoken

INFO: pip is looking at multiple versions of ragas to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.9/399.9 kB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 67.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.2/292.2 kB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 378.9/378.9 kB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 7.6

# 1 - Generating answer from Halubench dataset

In [ ]:
import pandas as pd
from datasets import Dataset, Features, Sequence, Value
from langchain_core.language_models import BaseLanguageModel
from langchain_core.embeddings import Embeddings
from langchain_community.llms import Ollama
from langchain_community.embeddings import HuggingFaceEmbeddings
from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy, answer_correctness
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

# Initialize Ollama LLM and Embeddings
llm_ans: BaseLanguageModel = Ollama(model="mistral", temperature = 0.1)
embeddings: Embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in _VertexAIBase has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in _VertexAICommon has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/ragas/metrics/__init__.py:4: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Load the dataset from a local CSV file
df = pd.read_csv('halubench_short.csv')

# Ensure the dataset has the required columns: 'question', 'contexts', 'ground_truth'
required_columns = ['question', 'contexts', 'ground_truth']
if not all(col in df.columns for col in required_columns):
    raise ValueError(f"The dataset must contain these columns: {required_columns}")

# Convert contexts column to a list of strings if it's not already
df['contexts'] = df['contexts'].apply(lambda x: x if isinstance(x, list) else [x])

In [ ]:
# Function to generate answer using Mistral
def generate_answer(question, context):
    prompt = f"Context: {context}\n\nQuestion: {question}\n\nProvide a concise answer in one sentence only:"
    response = llm_ans.invoke(prompt)
    return response.strip()

In [ ]:
# Generate answers using llm = Mistral
df['answer'] = df.apply(lambda row: generate_answer(row['question'], row['contexts']), axis=1)


In [ ]:
# Select required columns for the Hugging Face dataset
df_hf = df[required_columns + ['answer']]

# Define the schema for the Hugging Face dataset
features = Features({
    'question': Value('string'),
    'contexts': Sequence(Value('string')),
    'ground_truth': Value('string'),
    'answer': Value('string')
})


In [ ]:
# Save the DataFrame with generated answers to a CSV file
output_csv_path = 'halubench_short_answers_langchain_ollama_colab.csv'
df.to_csv(output_csv_path, index=False)

# S/S 2 - RAGAS Evaluation (Don't use this)

In [ ]:
import subprocess
process = subprocess.Popen("ollama serve", shell=True)
# credit: https://stackoverflow.com/questions/78437376/run-ollama-run-llama3-in-colab-raise-err-error-could-not-connect-to-ollama

# download the llama3 model
!ollama pull llama3.1 #download llama3 model
llm_eva: BaseLanguageModel = Ollama(model="llama3.1", temperature = 0.1, num_ctx = 8000)

pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠼ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠧ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠋ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠸ pulling manifest ⠸ pulling manifest ⠼ pulling manifest 
pulling 8eeb52dfb3bb...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 8eeb52dfb3bb...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 8eeb52dfb3bb...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 8eeb52dfb3bb...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 8eeb52dfb3bb...   0% ▕▏  17 MB/4.7 GB                  pulling manifest 
pulling 8eeb52dfb3bb...   3% ▕▏ 128 MB/4.7 GB                  pulling manifest 
pulling 8eeb52dfb3bb...   5% ▕▏ 226 MB/4.7 GB                  pulling manifest 
pulling 8eeb52dfb3bb...   6% ▕▏ 281 MB/4.7 GB                  pulling manifest 
pulling 8eeb52dfb3bb...   8% ▕▏ 385 MB/4.7 GB    

In [ ]:
# Convert the pandas DataFrame to a Hugging Face Dataset with the specified schema
dataset = Dataset.from_pandas(df_hf, features=features)

# Evaluate using Ragas with Ollama-LLama 3.1
results = evaluate(
    dataset=dataset,
    metrics=[faithfulness, answer_relevancy, answer_correctness],
    llm=llm_eva,
    embeddings=embeddings
)


Evaluating:   0%|          | 0/18 [00:00<?, ?it/s]

In [ ]:
# Print the results
print(results)

# Save the results to a CSV file
results.to_pandas().to_csv('halubench_short_result.csv', index=False)


{'faithfulness': nan, 'answer_relevancy': 0.7379, 'answer_correctness': 0.8994}


In [ ]:
df.head()

,Unnamed: 0,id,contexts,question,ground_truth,label,source_ds,answer
0,0,d3fb4c3c-d21b-480a-baa0-98d6d0d17c1d,[Hoping to rebound from the road loss to the C...,Which team scored the longest field goal kick ...,"['Rams', 'second', 'Marc Bulger', 'Kevin Curtis']",FAIL,DROP,Rams kicker Jeff Wilkins kicked the longest fi...
1,1000,25985014,[We investigated the role of surgical ablation...,Is ganglionated plexi ablation during Maze IV ...,No. GP ablation did not prove to be beneficial...,PASS,pubmedQA,"No, the study found that ganglionated plexi ab..."
2,2000,financebench_id_04451,[Index to Consolidated Financial Statements\nC...,What is the FY2021 operating cash flow ratio f...,0.25,FAIL,FinanceBench,The FY2021 operating cash flow ratio for CVS H...
3,3000,11904,"[passage 1:In short, whether mechanics and tec...",how do automotive technicians get paid,Automotive technicians can be paid in various ...,PASS,RAGTruth,Automotive technicians can be paid through var...
4,3905,halueval-5,"[Jonathan Stark (born April 3, 1971) is a form...",Which tennis player won more Grand Slam titles...,Henri Leconte won more Grand Slam titles.,FAIL,halueval,Jonathan Stark won more Grand Slam titles than...
